## This code implements the REINFORCE algorithm for the Cart Pole example provided by Open AI Gym.

This code is based on the Pytorch examples (https://github.com/pytorch/examples)

In [ ]:
import gym
import numpy as np
from itertools import count

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical


In [ ]:
seed = 1234
env = gym.make('CartPole-v1')
env.seed(seed)
torch.manual_seed(seed)

In [ ]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.input_layer = nn.Linear(4, 128)
        self.output = nn.Linear(128, 2)

        self.saved_log_probs = []
        self.rewards = []

    def forward(self, x):
        x = F.relu(self.input_layer(x))
        action_scores = self.output(x)
        return F.softmax(action_scores, dim=1)

In [ ]:
policy = Policy()
optimizer = optim.Adam(policy.parameters(), lr=1e-2)
eps = np.finfo(np.float32).eps.item()

In [ ]:
def select_action(state):
    state = torch.from_numpy(state).float().unsqueeze(0)
    probs = policy(state)
    m = Categorical(probs)
    action = m.sample()
    policy.saved_log_probs.append(m.log_prob(action))
    return action.item()


In [ ]:
def finish_episode():
    R = 0
    gamma=0.99
    policy_loss = []
    rewards = []
    for r in policy.rewards[::-1]:
        R = r + gamma * R
        rewards.insert(0, R)
    rewards = torch.tensor(rewards)
    rewards = (rewards - rewards.mean()) / (rewards.std() + eps)
    for log_prob, reward in zip(policy.saved_log_probs, rewards):
        policy_loss.append(-log_prob * reward)
    optimizer.zero_grad()
    policy_loss = torch.cat(policy_loss).sum()
    policy_loss.backward()
    optimizer.step()
    del policy.rewards[:]
    del policy.saved_log_probs[:]


In [ ]:
running_reward = 10
render=False
log_interval=10
for i_episode in count(1):
    state = env.reset()
    for t in range(10000):  # Don't infinite loop while learning
        action = select_action(state)
        state, reward, done, _ = env.step(action)
        if render:
            env.render()
        policy.rewards.append(reward)
        if done:
            break

    running_reward = running_reward * 0.99 + t * 0.01
    finish_episode()
    if i_episode % log_interval == 0:
        print('Episode {}\tLast length: {:5d}\tAverage length: {:.2f}'.format(
            i_episode, t, running_reward))
    if running_reward > env.spec.reward_threshold:
        print("Solved! Running reward is now {} and "
              "the last episode runs to {} time steps!".format(running_reward, t))
        break

Episode 10	Last length:     9	Average length: 10.38
Episode 20	Last length:    19	Average length: 10.85
Episode 30	Last length:    46	Average length: 13.77
Episode 40	Last length:     9	Average length: 15.31
Episode 50	Last length:   166	Average length: 20.69
Episode 60	Last length:    54	Average length: 28.59
Episode 70	Last length:   124	Average length: 34.04
Episode 80	Last length:    72	Average length: 40.26
Episode 90	Last length:    65	Average length: 45.72
Episode 100	Last length:    40	Average length: 47.37
Episode 110	Last length:    72	Average length: 47.15
Episode 120	Last length:    52	Average length: 48.14
Episode 130	Last length:   128	Average length: 55.04
Episode 140	Last length:   410	Average length: 73.21
Episode 150	Last length:   105	Average length: 93.85
Episode 160	Last length:    15	Average length: 87.71
Episode 170	Last length:    30	Average length: 81.45
Episode 180	Last length:    15	Average length: 75.62
Episode 190	Last length:    21	Average length: 70.66
Ep